In [1]:
import yfinance as yf
import requests
import pandas as pd
import datetime as dt
import time
from datetime import timedelta
from prod.model import *
from functools import reduce
from yahoo_fin import stock_info as si
import numpy as np
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import pygsheets


In [2]:
# inversion inicial en dolares
#initial_amount=5249000/777.18
initial_amount=5000000/777.18

# ponderacion del portafolio
ponderacion={'ETHO':4.6/100,
'ESGV':4.12/100,
'VT':12.16/100,
'XLK':15.16/100,
'VTI':19.61/100,
'IVV':11.42/100,
'QQQ':15.30/100,
'ESGU':4.79/100,
'SMH':6.72/100,
'ARKW':2.30/100,
'ARKK':2.13/100,
'ESGU':4.79/100,
'OTROS':2.23/100}

# portafolio en el que se invierte
tickers=["ESGV","ETHO","VT","XLK","VTI","IVV","QQQ","SMH","ARKW","ARKK","ESGU"]

In [3]:
dataframes=[]
for i in tickers:
       
    # Generamos el diccionario pada cada ticker
    string="{i}=ticker('{i}')".format(i=i)
    exec(string)
    
    string2="df_{i}={i}.getLatest()".format(i=i)
    exec(string2)
    
    string3="df_{i}['ticker']='{i}'".format(i=i)
    exec(string3)
    
    string4="dataframes.append(df_{i})".format(i=i)
    exec(string4)

# Concatenamos todos los dataframes en solo uno
day_display=reduce(lambda x, y: pd.concat([x, y]), dataframes)
day_display

08/13/2020
Creating ticker ...
08/13/2020
Creating ticker ...
08/13/2020
Creating ticker ...
08/13/2020
Creating ticker ...
08/13/2020
Creating ticker ...
08/13/2020
Creating ticker ...
08/13/2020
Creating ticker ...
08/13/2020
Creating ticker ...
08/13/2020
Creating ticker ...
08/13/2020
Creating ticker ...
08/13/2020
Creating ticker ...


,date,open,high,low,last_price,change,price_change,ticker
0,2020-08-13 13:30:00,61.840000,61.950100,61.669998,61.950,0.177878,0.110000,ESGV
0,2020-08-13 13:30:00,44.810001,44.931198,44.810001,44.890,0.178529,0.079999,ETHO
0,2020-08-13 13:30:00,81.699997,81.970001,81.629997,81.950,0.306001,0.250003,VT
0,2020-08-13 13:30:00,114.070000,114.850800,113.800003,114.825,0.661875,0.755000,XLK
0,2020-08-13 13:30:00,170.789993,171.678101,170.789993,171.645,0.500619,0.855007,VTI
0,2020-08-13 13:30:00,338.000000,339.480011,337.809998,339.435,0.424556,1.435000,IVV
0,2020-08-13 13:30:00,272.640015,274.679993,272.390015,274.545,0.698718,1.904985,QQQ
0,2020-08-13 13:30:00,172.410004,172.600006,170.850006,171.860,-0.319009,-0.550004,SMH
0,2020-08-13 13:30:00,99.150002,100.345001,98.922401,100.260,1.119514,1.109998,ARKW
0,2020-08-13 13:30:00,84.510002,86.389999,84.510002,86.270,2.082591,1.759998,ARKK


In [4]:
# agregamos una columna de ponderacion de cada instrumento
day_display["pond"]=day_display["ticker"].apply(lambda x: ponderacion.get(x))

# Extraemos los montos que no tuvieron pnderacion
not_considered_per=1-sum(day_display["pond"])

# Guardamos lo que no se invirtio.. no se considera en la estimacion
rest=initial_amount*not_considered_per
invested=initial_amount-rest

# Agregamos columnas de cantidad inicial invertida y no invertida independiente del fondo
day_display["initial_invested_usd"]=initial_amount
day_display["real_invested"]=invested
day_display["not_invested"]=rest

# Calculamos lo que se invirtio en cada instrumento
day_display["invested"]=day_display["pond"].apply(lambda x: x*initial_amount)

# Calculamos las ganancias o perdidas
day_display["profit"]=(day_display["change"]/100)*(day_display["invested"])

# Calculamos el nuevo valor de la cantidad invertida en el instrumento 
day_display["estimated"]=day_display["invested"]+day_display["profit"]

# Agregamos una columna con el resultado final de lo invertido
day_display['total_result']=sum(day_display["estimated"])-sum(day_display["invested"])

In [5]:
day_display

,date,open,high,low,last_price,change,price_change,ticker,pond,initial_invested_usd,real_invested,not_invested,invested,profit,estimated,total_result
0,2020-08-13 13:30:00,61.840000,61.950100,61.669998,61.950,0.177878,0.110000,ESGV,0.0412,6433.516045,6324.789624,108.726421,265.060861,0.471485,265.532346,30.457168
0,2020-08-13 13:30:00,44.810001,44.931198,44.810001,44.890,0.178529,0.079999,ETHO,0.0460,6433.516045,6324.789624,108.726421,295.941738,0.528340,296.470078,30.457168
0,2020-08-13 13:30:00,81.699997,81.970001,81.629997,81.950,0.306001,0.250003,VT,0.1216,6433.516045,6324.789624,108.726421,782.315551,2.393896,784.709447,30.457168
0,2020-08-13 13:30:00,114.070000,114.850800,113.800003,114.825,0.661875,0.755000,XLK,0.1516,6433.516045,6324.789624,108.726421,975.321032,6.455402,981.776434,30.457168
0,2020-08-13 13:30:00,170.789993,171.678101,170.789993,171.645,0.500619,0.855007,VTI,0.1961,6433.516045,6324.789624,108.726421,1261.612496,6.315869,1267.928365,30.457168
0,2020-08-13 13:30:00,338.000000,339.480011,337.809998,339.435,0.424556,1.435000,IVV,0.1142,6433.516045,6324.789624,108.726421,734.707532,3.119246,737.826779,30.457168
0,2020-08-13 13:30:00,272.640015,274.679993,272.390015,274.545,0.698718,1.904985,QQQ,0.1530,6433.516045,6324.789624,108.726421,984.327955,6.877678,991.205633,30.457168
0,2020-08-13 13:30:00,172.410004,172.600006,170.850006,171.860,-0.319009,-0.550004,SMH,0.0672,6433.516045,6324.789624,108.726421,432.332278,-1.379179,430.953099,30.457168
0,2020-08-13 13:30:00,99.150002,100.345001,98.922401,100.260,1.119514,1.109998,ARKW,0.0230,6433.516045,6324.789624,108.726421,147.970869,1.656555,149.627424,30.457168
0,2020-08-13 13:30:00,84.510002,86.389999,84.510002,86.270,2.082591,1.759998,ARKK,0.0213,6433.516045,6324.789624,108.726421,137.033892,2.853856,139.887748,30.457168


In [6]:
def save_gsheets(page,dataframe):
    gc = pygsheets.authorize(service_file='key.json')

    #open the google spreadsheet 
    sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1z7zzHgM6t4ySUkPZe5BdFsBvTD6RxsfSdKEnqWU1a2Q/edit#gid=67598709')

    #select the first sheet 
    wks = sh[page]

    #update the first sheet with df, starting at cell B2. 
    wks.set_dataframe(dataframe, 'A1')

In [7]:
save_gsheets(0,day_display)